# NasNet Dog breed classification

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from fastai.conv_learner import *
PATH = "data/dogbreed/"
sz=224; bs=48

In [3]:
def nasnet(pre): return nasnetalarge(pretrained = 'imagenet' if pre else None)
model_features[nasnet]=4032*2

In [4]:
label_csv = f'{PATH}labels.csv'
n = len(list(open(label_csv))) - 1
val_idxs = get_cv_idxs(n)

In [5]:
stats = ([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
tfms = tfms_from_stats(stats, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
#data = ImageClassifierData.from_paths(PATH, tfms=tfms, bs=bs)

#tfms = tfms_from_model(arch, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv', test_name='test',
                                    val_idxs=val_idxs, suffix='.jpg', tfms=tfms, bs=bs)

In [6]:
learn = ConvLearner.pretrained(nasnet, data, precompute=True, xtra_fc=[], ps=0.5)

Downloading: "http://webia.lip6.fr/~cadene/Downloads/pretrained-models.pytorch/nasnetalarge-dc8c1432.pth" to /home/nbuser/.torch/models/nasnetalarge-dc8c1432.pth
100%|██████████| 356078260/356078260 [00:22<00:00, 15556424.59it/s]


100%|██████████| 216/216 [30:30<00:00,  8.47s/it]


In [7]:
%time learn.fit(1e-2, 5)

HBox(children=(IntProgress(value=0, description='Epoch', max=5), HTML(value='')))

[ 0.       0.72086  0.56786  0.88386]                        
[ 1.       0.42238  0.56715  0.87708]                        
[ 2.       0.29238  0.53173  0.89743]                        
[ 3.       0.2254   0.56657  0.89113]                        
[ 4.       0.20605  0.57333  0.8903 ]                        

CPU times: user 14.6 s, sys: 6.52 s, total: 21.1 s
Wall time: 52.9 s


In [8]:
learn.precompute=False
learn.bn_freeze=True

In [ ]:
%time learn.fit(1e-2, 3, cycle_len=1)

HBox(children=(IntProgress(value=0, description='Epoch', max=3), HTML(value='')))

[ 0.       0.29305  0.52536  0.8932 ]                        
[ 1.       0.25835  0.53723  0.89452]                        
[ 2.       0.24882  0.52227  0.89694]                        

CPU times: user 1h 13min 57s, sys: 5min 55s, total: 1h 19min 52s
Wall time: 1h 14min 49s


In [ ]:
%time learn.fit(1e-2, 3, cycle_len=1, cycle_mult=2)

HBox(children=(IntProgress(value=0, description='Epoch', max=7), HTML(value='')))

[ 0.       0.23405  0.51515  0.89902]                        
 16%|█▌        | 27/171 [03:11<16:59,  7.08s/it, loss=0.258]

In [ ]:
learn.save('nas_pre')

In [ ]:
def freeze_to(m, n):
    c=children(m[0])
    for l in c:     set_trainable(l, False)
    for l in c[n:]: set_trainable(l, True)

In [ ]:
freeze_to(learn.model, 17)

In [ ]:
learn.fit([1e-5,1e-4,1e-2], 1, cycle_len=1)

In [ ]:
learn.save('nas')

In [ ]:
def get_data(sz, bs):
    tfms = tfms_from_stats(stats, sz, aug_tfms=transforms_side_on, max_zoom=1.1)
    data = ImageClassifierData.from_csv(PATH, 'train', f'{PATH}labels.csv', test_name='test',
                                    val_idxs=val_idxs, suffix='.jpg', tfms=tfms, bs=bs)
    return data if sz>300 else data.resize(340, 'tmp')

In [ ]:
learn.set_data(get_data(350, bs))
learn.freeze()